In [122]:
import scrapy
import random
import pymongo
import pandas as pd
from bs4 import BeautifulSoup
from pymongo import TEXT
import pandas as pd
import datetime
import sys
sys.path.append(".")
sys.path.append("/home/ubuntu/fch-virus-combat/Spider")

In [127]:
import requests
from settings import Settings
from helper import quick_list_collection
from processing_harianmetro import Processing_HarianMetro
from processing_thestar import Processing_TheStar
MONGO_URI = "localhost:27017"
MONGO_DB = "news"
RAW_COLLECTION = Settings.METRO_RAW_MONGO_COLLECTION
# RAW_COLLECTION = Settings.THESTAR_RAW_MONGO_COLLECTION
# RAW_COLLECTION = Settings.SEBENARNYA_RAW_MONGO_COLLECTION
PRO_COLLECTION = "sebenarnya_pro"
PRO_COLLECTION = "hmetro_pro"
# PRO_COLLECTION = "thestar_pro"
client = pymongo.MongoClient(MONGO_URI)
db = client[MONGO_DB]
raw_coll = db[RAW_COLLECTION]
pro_coll = db[PRO_COLLECTION]
# quick_list_collection()

def check_duplicates(raw_coll):
    urls = [j.get("url") for j in raw_coll.find()]
    sd = [j.get("scrape_date") for j in raw_coll.find()]
    df = pd.DataFrame(dict(url = urls, sd = sd))
    z = df["url"].value_counts()

    df = df.set_index("url")
    df["z"] = z
    return (df.z >= 2).mean()

## remove duplicates in mongoshell
'''
use news
db.thestar_raw.find({}, {url:1}).sort({_id:1}).forEach(function(doc){
    db.thestar_raw.remove({_id:{$gt:doc._id}, url:doc.url});
})
'''
###
check_duplicates(raw_coll)

0.0

In [94]:
u = "https://www.thestar.com.my/news/nation/2020/04/10/dear-yb-enough-with-the-photo-ops-walk-the-mco-talk"
# u = "https://www.thestar.com.my/news/nation/2020/04/11/covid-19-large-scale-community-screening-tests-conducted-in-hulu-langat"
d = raw_coll.find_one({"url": u})

In [101]:
d = raw_coll.find_one( sort=[( '_id', -1 )]) 
d.get("scrape_date")

datetime.datetime(2020, 4, 11, 10, 5, 0, 838000)

In [99]:
Processing_TheStar(d).pro
# d

AttributeError: 'NoneType' object has no attribute 'get_text'

In [98]:
soup = BeautifulSoup(d.get("content_html"))

## ElasticSearch

In [113]:
from elasticsearch import Elasticsearch
es = Elasticsearch()
u = "https://www.thestar.com.my/news/nation/2020/03/23/covid-19-current-situation-in-malaysia-updated-daily"
search_param = {
    "query": {
        "match": {
            "url": u
        }
    }
}
response = es.search(index="my_index", body=search_param)

In [118]:
es.delete_by_query(index='all_news', body={"query": {"match_all": {}}})

{'took': 24,
 'timed_out': False,
 'total': 1,
 'deleted': 1,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [115]:
# response

In [109]:
keyword = "test"
r = search(keyword)
ddf_fna, n_fna, mscore_fna = get_top_news(r, category = "FakeNewsAlert")
ddf_news, n_news, mscore_news = get_top_news(r, category = "News")
predict(n_fna, n_news, mscore_fna, mscore_news)

1

In [39]:
raw_coll.find_one({"url": 0})

In [111]:
ddf_news

{'TheStar':                                                title  \
 9        Data of those who took Covid-19 test leaked   
 8               Hospitals to join in WHO drug trials   
 5  Hua Zong urges Putrajaya to ensure enough supp...   
 
                                                  url category   source  \
 9  https://www.thestar.com.my/news/nation/2020/04...     News  TheStar   
 8  https://www.thestar.com.my/news/nation/2020/04...     News  TheStar   
 5  https://www.thestar.com.my/news/nation/2020/04...     News  TheStar   
 
       score  
 9  4.622537  
 8  4.662876  
 5  4.959969  }

In [43]:
r = search("stok beras 2.5")
df = pd.DataFrame(r)

In [46]:
df[["source", "category"]].unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [44]:
df

,title,url,category,source,score
0,Stok Beras Negara Hanya Mampu Bertahan Selama ...,https://sebenarnya.my/stok-beras-negara-hanya-...,FakeNewsAlert,sebenarnya,27.278168
1,Penduduk keluar secara berkala,https://www.hmetro.com.my/mutakhir/2020/04/564...,News,HarianMetro,10.438990
2,"'Tak guna bertekak, baik tolong orang' - Hattan",https://www.hmetro.com.my/rap/2020/04/563691/t...,News,HarianMetro,10.438990
3,"Hospital Pontian, Johor Kekurangan Stok Topeng...",https://sebenarnya.my/hospital-pontian-johor-k...,FakeNewsAlert,sebenarnya,9.621781
4,Kedah tetap 4 kadar zakat fitrah,https://www.hmetro.com.my/mutakhir/2020/04/564...,News,HarianMetro,8.118494
5,Peniaga kuih hanya mampu pasrah,https://www.hmetro.com.my/mutakhir/2020/04/562...,News,HarianMetro,7.275163
6,"No decision on Hajj pilgrimage from Saudi yet,...",https://www.thestar.com.my/news/nation/2020/04...,News,TheStar,6.300395
7,"No decision on Hajj pilgrimage from Saudi yet,...",https://www.thestar.com.my/news/nation/2020/04...,News,TheStar,6.300395
8,AIM sumbang pek makanan asas,https://www.hmetro.com.my/mutakhir/2020/04/562...,News,HarianMetro,5.683450
9,USM edar kit barangan keperluan asas,https://www.hmetro.com.my/mutakhir/2020/04/562...,News,HarianMetro,5.416268


In [34]:
categories

['FakeNewsAlert', 'FakeNewsAlert', 'FakeNewsAlert']

['sebenarnya', 'sebenarnya', 'sebenarnya', 'sebenarnya', 'sebenarnya']

In [ ]:
# from elasticsearch import Elasticsearch
# from datetime import datetime
# INDEX_NAME = 'news'
# ec_conn= Elasticsearch('http://127.0.0.1:9200')
# ec_conn

# ec_conn.search(index='news', body = {'query':{"match": {"title":"artikel", "content_text": "artikel"}}})


In [ ]:
d = raw_coll.find_one({"url":"https://www.hmetro.com.my/arena/2020/04/564665/penyambungan-segera-liga-m-samar"})
d = pro_coll.find_one()
# Processing_HarianMetro(d).pro
d

In [ ]:
x = list(pro_coll.find({}, {"news_date": 1}))
df = pd.DataFrame(x)
df["news_date"].min(), df["news_date"].max()

In [119]:
def reset_raw_collection():
    MONGO_URI = Settings.MONGO_URI
    MONGO_DB = Settings.MONGO_DB
    RAW_COLLECTION_LIST = [Settings.METRO_RAW_MONGO_COLLECTION, Settings.THESTAR_RAW_MONGO_COLLECTION, Settings.SEBENARNYA_RAW_MONGO_COLLECTION]
    client = pymongo.MongoClient(MONGO_URI)
    db = client[MONGO_DB]
    for RAW_COLLECTION in RAW_COLLECTION_LIST:
        raw_coll = db[RAW_COLLECTION]
        update = { "$unset": { "processed_date": None } }
        raw_coll.update_many({}, update)
    print("Done: Reset processed_date...")
        
reset_raw_collection()


Done: Reset processed_date...


In [ ]:
x = list(raw_coll.find({}, {"news_date": 1}))

In [ ]:
pd.DataFrame(x)["news_date"].min(), pd.DataFrame(x)["news_date"].max()

In [ ]:
# d = raw_coll.find_one({"url":"https://www.thestar.com.my/sport/badminton/2020/04/08/could-thomas-uber-cup-face-same-fate-as-olympics"})
# url = "https://www.thestar.com.my/news/nation/2020/04/07/khaled-nordin-post-mco-039exit-strategy039-needed-for-malaysia"
# d = raw_coll.find_one({"url":url})

In [ ]:
update = { "$unset": { "processed_date": None } }
raw_coll.update_many({}, update)